In [ ]:
!pip install scikit-learn
!pip install nltk
!pip install pandas

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

dataset_url = "http://snap.stanford.edu/data/amazon/productGraph/kcore_5.json.gz"
save_path = "kcore_5.json.gz"

# Download the 5-core dataset
urllib.request.urlretrieve(dataset_url, save_path)

print("Dataset downloaded successfully.")

In [22]:
import urllib.request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import gzip
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import pandas as pd
import json
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

ModuleNotFoundError: No module named 'textblob'

In [5]:
data_file = "kcore_5.json\kcore_5.json"
num_lines = 10000

# Create an empty list to store the JSON objects
json_data = []

# Read the JSON file and load the JSON objects
with open(data_file, "r") as f:
    for _ in range(num_lines):
        line = f.readline()
        json_obj = json.loads(line)
        json_data.append(json_obj)

# Create a DataFrame from the JSON data
df = pd.DataFrame(json_data)

# Print the DataFrame
print(df)

          reviewerID        asin               reviewerName helpful  \
0      ACNGUPJ3A3TM9  0000013714                        GCM  [0, 0]   
1     A2SUAM1J3GNN3B  0000013714                J. McDonald  [2, 3]   
2      APOZ15IEYQRRR  0000013714                  maewest64  [0, 0]   
3      AYEDW3BFK53XK  0000013714                      Missb  [0, 0]   
4     A1KLCGLCXYP1U1  0000013714        Paul L "Paul Lytle"  [0, 0]   
...              ...         ...                        ...     ...   
9995  A32UCGRPBE03R3  0002558033                  Greyhound  [1, 1]   
9996   AIMQH7610W8NK  0002558033             jared matthews  [4, 5]   
9997   AVTCCT91Q6O5N  0002558033  Jason A. Carson "Tai Guy"  [1, 4]   
9998   AYI3EMZAXS6E8  0002558033                 Jason Mack  [0, 0]   
9999  A13L24Q3OKM0VK  0002558033                Juan Castro  [0, 8]   

                                             reviewText  overall  \
0     We use this type of hymnal at church.  I was l...      4.0   
1     I bou

In [15]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,preprocessed_review
0,ACNGUPJ3A3TM9,0000013714,GCM,"[0, 0]",We use this type of hymnal at church. I was l...,4.0,Nice Hymnal,1386028800,"12 3, 2013",use type hymnal church looking one however n't...
1,A2SUAM1J3GNN3B,0000013714,J. McDonald,"[2, 3]",I bought this for my husband who plays the pia...,5.0,Heavenly Highway Hymns,1252800000,"09 13, 2009",bought husband play piano wonderful time playi...
2,APOZ15IEYQRRR,0000013714,maewest64,"[0, 0]",This is a large size hymn book which is great ...,5.0,Awesome Hymn Book,1362787200,"03 9, 2013",large size hymn book great able see song note ...
3,AYEDW3BFK53XK,0000013714,Missb,"[0, 0]",We use this hymn book at the mission. It has ...,5.0,Hand Clapping Toe Tapping Oldies,1325462400,"01 2, 2012",use hymn book mission well-loved oldie time go...
4,A1KLCGLCXYP1U1,0000013714,"Paul L ""Paul Lytle""","[0, 0]","One review advised this book was large print, ...",3.0,Misleading,1376092800,"08 10, 2013",one review advised book large print however n'...


In [13]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())
    
    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into a preprocessed text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [14]:
# Preprocess the review texts
df['preprocessed_review'] = df['reviewText'].apply(preprocess_text)

In [20]:
# Example usage
# Assuming you already have a DataFrame called 'df' with a column named 'preprocessed_review'

# Initialize the TF-IDF vectorizer.
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the preprocessed reviews
tfidf_matrix = vectorizer.fit_transform(df['preprocessed_review'])

# Get the feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Print the TF-IDF DataFrame
print(tfidf_df)

       00  000  0001714422  000th   01  016   02   03  0394826922   04  ...  \
0     0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
1     0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
2     0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
3     0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
4     0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
...   ...  ...         ...    ...  ...  ...  ...  ...         ...  ...  ...   
9995  0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
9996  0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
9997  0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
9998  0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   
9999  0.0  0.0         0.0    0.0  0.0  0.0  0.0  0.0         0.0  0.0  ...   

      zoo  zookeeper  zoologist  zoom  zoomed  zoos

In [23]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    return sentiment

# Apply sentiment analysis to the preprocessed reviews
df['sentiment_score'] = df['preprocessed_review'].apply(analyze_sentiment)

# Classify the sentiment based on the sentiment score
df['sentiment'] = df['sentiment_score'].apply(lambda score: 'Positive' if score > 0 else 'Negative' if score < 0 else 'Neutral')

# Print the DataFrame with sentiment scores and classifications
print(df[['preprocessed_review', 'sentiment_score', 'sentiment']])

NameError: name 'TextBlob' is not defined